In [133]:
import pandas as pd
import features
import glob,re,pickle,os,yaml,datetime

In [134]:
debut=datetime.datetime.now()
filePrefix="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/LongitudinalesRnd/"
filePrefix="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/lexique3/"
sampleFiles=glob.glob(filePrefix+"Longitudinal*.csv")

In [135]:
def prefixEchantillon(numero):
    candidats=[]
    for sample in sampleFiles:
#        m=re.match(ur"^.*/(Longitudinal-%s-T\d+-F\d+)%s-paradigmes\.csv"%(numero,casesType),sample)
        m=re.match(ur"^.*/(Longitudinal-Lexique3-%s-T\d+-F\d+)%s-paradigmes\.csv"%(numero,casesType),sample)
        if m:
            candidats.append(m.group(1))
    if len(candidats)==1:
        return candidats[0]
    else:
        print "PB pas de nom unique correspondant",candidats

In [136]:
numeroEchantillon="00"
sampleType="-X"
# casesType="-X-Morphomes"
casesType="-X"
etapeSwim="-Swim2"
samplePrefix=prefixEchantillon(numeroEchantillon)
initialFile=filePrefix+samplePrefix+"-X-paradigmes.csv"
analysisPrefix=filePrefix+samplePrefix+casesType
predictionsFile=analysisPrefix+"-paradigmes%s.csv"%etapeSwim
print predictionsFile
referenceFile="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/MGC-171229-Verbes3.pkl"

/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/lexique3/Longitudinal-Lexique3-00-T150000000-F73539-X-paradigmes-Swim2.csv


## Réglages de la variante phonologique

In [137]:
phonologicalMap=sampleType.strip("-")
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [138]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [139]:
dierese={"j":"ij", "w":"uw","H":"yH","i":"ij","u":"uw","y":"yH"}

In [140]:
def checkFrench(prononciation):
    glide2voyelle={"j":"i","w":"u","H":"y"}
    if prononciation:
        if "," in prononciation:
            prononciations=prononciation.split(",")
            setPrononciations=set()
            for element in prononciations:
                setPrononciations.add(checkFrench(element))
            result=",".join(list(setPrononciations))
        else:
            result=recoder(prononciation)
            result=result.replace("Jj","J")
            m=re.match(ur"^(.*[^ieèEaOouy926êôâ])([jwH])$",result)
            if m:
                print ("pb avec un glide final", prononciation)
                voyelle=glide2voyelle[m.group(2)]
                result=m.group(1)+voyelle
            m=re.match(ur"(.*[ptkbdgfsSvzZ][rl])([jwH])(.*)",result)
            if m:
                n=re.search(ur"[ptkbdgfsSvzZ][rl](wa|Hi|wê)",result)
                if not n:
                    glide=m.group(2)
                    result=m.group(1)+dierese[glide]+m.group(3)
            m=re.match(ur"(.*)([iuy])([ieEaOouy].*)",result)
            if m:
                glide=m.group(2)
                result=m.group(1)+dierese[glide]+m.group(3)
            result=result.replace("jj","ij")
    else:
        result=prononciation
    return result

In [141]:
print checkFrench(u"kopjjô")

kopijô


## Formes de l'échantillon

In [142]:
initialParadigmes=pd.read_csv(initialFile,sep=";",encoding="utf8")
del initialParadigmes[u"Unnamed: 0"]
initialParadigmes=initialParadigmes.dropna(axis=1,how='all')
initialParadigmesColumns=initialParadigmes.columns.tolist()
listeLexemes=initialParadigmes["lexeme"].tolist()
nbLexemes=len(listeLexemes)

In [143]:
initialForms=pd.melt(initialParadigmes[initialParadigmes["lexeme"].isin(listeLexemes)],id_vars=["lexeme"]).dropna()
initialForms["lexeme-case"]=initialForms["lexeme"]+"-"+initialForms["variable"]
initialForms.drop(labels=["lexeme","variable"],axis=1,inplace=True)
initialForms.set_index(["lexeme-case"],inplace=True)
initialFormsIndex=initialForms.index.tolist()

## Formes de l'échantillon avec les prédictions

In [144]:
predictedParadigmes=pd.read_csv(predictionsFile,sep=";",encoding="utf8")
del predictedParadigmes[u"Unnamed: 0"]
predictedParadigmes=predictedParadigmes.loc[:,predictedParadigmes.columns.isin(initialParadigmesColumns)].dropna(axis=1,how='all')
if listeLexemes!=predictedParadigmes["lexeme"].tolist():
    print "PB avec la liste des lexèmes prédits"
if set(initialParadigmesColumns)!=set(predictedParadigmes.columns.tolist()):
    print "PB avec la liste des cases prédites"
    print predictedParadigmes.columns.tolist()
    print initialParadigmesColumns

In [145]:
predictedForms=pd.melt(predictedParadigmes[predictedParadigmes["lexeme"].isin(listeLexemes)],id_vars=["lexeme"]).dropna()
predictedForms["lexeme-case"]=predictedForms["lexeme"]+"-"+predictedForms["variable"]
predictedForms.drop(labels=["lexeme","variable"],axis=1,inplace=True)
predictedForms.set_index(["lexeme-case"],inplace=True)
predictedFormsIndex=predictedForms.index.tolist()

In [146]:
predictedForms

,value
lexeme-case,
abaisser-ai1P,abEsam
abandonner-ai1P,abâdOnam
abasourdir-ai1P,abazurdim
abattre-ai1P,abatim
abdiquer-ai1P,abdikam
aberrer-ai1P,abEram
abhorrer-ai1P,abOram
abjurer-ai1P,abZyram
abolir-ai1P,abOlim


## Formes de référence

In [147]:
# with open(referenceFile,"rb") as input:
#     lexiqueGold=pickle.load(input)

lexiqueGold=pd.read_pickle(referenceFile)

In [148]:
lexiqueGold[lexiqueGold["lexeme"]=="copier"]

,ortho,phono,ext,cs,ms,vs,lexeme,freq,prob,case
86040,copia,kOpja,NaN,V,3S,ai,copier,8.000061e+06,2.317306e-07,ai3S
86043,copiai,kOpjE,NaN,V,1S,ai,copier,7.000007e+06,2.027630e-07,ai1S
86044,copiaient,kOpjE,"t""",V,3P,ii,copier,2.800003e+07,8.110518e-07,ii3P
86045,copiais,kOpjE,"z""",V,1S,ii,copier,3.614247e+07,1.046907e-06,ii1S
86046,copiais,kOpjE,"z""",V,2S,ii,copier,8.575646e+05,2.484031e-08,ii2S
86047,copiait,kOpjE,"t""",V,3S,ii,copier,4.800014e+07,1.390377e-06,ii3S
86048,copiant,kOpjâ,"t""",V,,pP,copier,4.400003e+07,1.274510e-06,pP
86049,copias,kOpja,"z""",V,2S,ai,copier,2.039116e-01,5.906524e-15,ai2S
86050,copiasse,kOpjas,@,V,1S,is,copier,1.500807e-01,4.347254e-15,is1S
86051,copiassent,kOpjas,"@t""",V,3P,is,copier,2.523647e-01,7.310024e-15,is3P


In [149]:
'''Rectifications phonologiques'''
lexiqueGold["phono"]=lexiqueGold["phono"].apply(lambda x: checkFrench(x))
completeParadigmes=pd.pivot_table(lexiqueGold, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()

'''Identification des cases présentes initialement'''
completeParadigmes=completeParadigmes.loc[:,completeParadigmes.columns.isin(initialParadigmesColumns)]

'''Mise en liste des formes de références'''
goldTestForms=pd.melt(completeParadigmes[completeParadigmes["lexeme"].isin(listeLexemes)],id_vars=["lexeme"]).dropna()
goldTestForms["lexeme-case"]=goldTestForms["lexeme"]+"-"+goldTestForms["case"]
goldTestForms.drop(labels=["lexeme","case"],axis=1,inplace=True)
goldTestForms.set_index(["lexeme-case"],inplace=True)

'''Extraction des formes de références pertinentes'''
goldForms=goldTestForms.loc[~goldTestForms.index.isin(initialFormsIndex)]
goldFormsIndex=goldForms.index.tolist()

In [150]:
def countSplits(dfForms):
    dfForms.loc[:,"split"]=dfForms.loc[:,"value"].str.split(",")
    return dfForms["split"].str.len().sum()


In [151]:
'''Soustraire les formes initiales des prédictions'''
finalForms=predictedForms.loc[~predictedForms.index.isin(initialFormsIndex)]
finalFormsIndex=finalForms.index.tolist()

In [152]:
finalForms["value"]=finalForms["value"].apply(lambda x: checkFrench(x))

/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Est-ce que la surabondance est gérée entre l'échantillon et la référence ?

In [153]:
nbInitialForms=countSplits(initialForms)
nbGoldForms=countSplits(goldForms)
print nbGoldForms, "formes à prédire à partir de",nbInitialForms,"formes de",nbLexemes,"lexèmes"

197115 formes à prédire à partir de 73539 formes de 5384 lexèmes


In [154]:
'''Calculer les sur/sous-générations'''
underGeneration=goldForms.loc[~goldForms.index.isin(finalFormsIndex)]
overGeneration=finalForms.loc[~finalForms.index.isin(goldFormsIndex)]
print (countSplits(underGeneration),countSplits(overGeneration))

(33753, 3676)


In [155]:
'''Réduire les prédictions et la référence aux cases communes'''
compareForms=finalForms.loc[finalForms.index.isin(goldFormsIndex)].copy()
actualForms=goldForms.loc[goldForms.index.isin(finalFormsIndex)]
len(compareForms),len(actualForms)

(162929, 162929)

In [156]:
'''Créer un tableau pour les comparaisons'''
compareForms.loc[:,"right"]=actualForms.loc[:,"value"]
countSplits(compareForms)

163285

In [157]:
'''Séparer les cases identiques des cases différentes'''
sameForms=compareForms[compareForms["value"]==compareForms["right"]]
diffForms=compareForms[compareForms["value"]!=compareForms["right"]]

In [158]:
print (countSplits(sameForms))
print (countSplits(diffForms))

161751
1534


In [159]:
features.add_config('../bdlexique.ini')
fs=features.FeatureSystem('phonemes')

def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

class formesPatron:
    '''
    Accumulateur de formes correspondant à un patron pour calcul de la Généralisation Minimale (cf. MGL)
    '''
    def __init__(self):
        self.formes=[]

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterForme(self,forme):
        self.formes.append(forme)
        
    def calculerGM(self):
        minLongueur=len(min(self.formes, key=len))
        maxLongueur=len(max(self.formes, key=len))
#        if debug: print (minLongueur, maxLongueur, file=logfile)
        positions=[]
        if maxLongueur>minLongueur:
            positions.append("*")
        for i in xrange(minLongueur, 0, -1):
            phonemes=set([x[-i] for x in self.formes])
#            if debug: print (phonemes, file=logfile)
            if "." in phonemes:
                positions.append(".")
            else:
                print u"phonèmes",phonemes
                positions.append("".join(fs.lattice[phonemes].extent))
        return patron2regexp(positions)

class pairePatrons:
    '''
    Accumulateur de triplets (f1,f2,patron) correspondant à une paire pour calcul des Généralisations Minimales (cf. MGL)
    '''
    def __init__(self,case1,case2):
        self.patrons1={}
        self.patrons2={}
        self.case1=case1
        self.case2=case2

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterFormes(self,forme1,forme2,patron):
#        print (forme1,forme2,patron, file=logfile)
        patron12=patron
        (pat1,pat2)=patron.split("-")
        patron21=pat2+"-"+pat1
#        print (patron12,patron21, file=logfile)
        if not patron12 in self.patrons1:
            self.patrons1[patron12]=formesPatron()
        self.patrons1[patron12].ajouterForme(forme1)
        if not patron21 in self.patrons2:
            self.patrons2[patron21]=formesPatron()
        self.patrons2[patron21].ajouterForme(forme2)
        
        
    def calculerGM(self):
        resultat1={}
        for patron in self.patrons1:
#            if debug: print ("patron1", patron, file=logfile)
            resultat1[patron]=self.patrons1[patron].calculerGM()
        resultat2={}
        for patron in self.patrons2:
#            if debug: print ("patron2", patron, file=logfile)
            resultat2[patron]=self.patrons2[patron].calculerGM()
        return (resultat1,resultat2) 

class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
#        else:
#            if debug: print ("le numéro de forme n'est pas dans [1,2]",n, file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
        self.classeCF={}
        self.nbClasseCF={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeFormeCF=[]
        regleFormeCF=""
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            filterF1=".*"+patron.split("-")[0]+"$"
            if re.match(filterF1,forme1):
                classeFormeCF.append(patron)
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleFormeCF=patron
            filterF1=self.patrons[patron]
            if re.match(filterF1,forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseFormeCF=", ".join(classeFormeCF)
        if not idClasseFormeCF in self.classeCF:
            self.classeCF[idClasseFormeCF]={}
            self.nbClasseCF[idClasseFormeCF]=0
        if not regleFormeCF in self.classeCF[idClasseFormeCF]:
            self.classeCF[idClasseFormeCF][regleFormeCF]=0
        self.nbClasseCF[idClasseFormeCF]+=1
        self.classeCF[idClasseFormeCF][regleFormeCF]+=1
        
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme,contextFree=False):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if contextFree:
                filterF1=".*"+patron.split("-")[0]+"$"
            else:
                filterF1=self.patrons[patron]
            if re.match(filterF1,forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if contextFree:
                nbClasse=self.nbClasseCF
                classe=self.classeCF
            else:
                nbClasse=self.nbClasse
                classe=self.classe
            if idClasseForme in nbClasse:
                nTotal=nbClasse[idClasseForme]
                for patron in classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(classe[idClasseForme][patron])/nTotal
            else:
#                if debug: 
#                    print (forme, file=logfile)
#                    print ("pas de classe",idClasseForme, file=logfile)
#                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
#        else:
#            if debug:
#                print (forme, file=logfile) 
#                print ("pas de patron", file=logfile)
        return sortieForme
        

def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)



def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


In [160]:
validPhonemes=list(fs.supremum.concept.extent)
validPhonemes

[u'p',
 u't',
 u'k',
 u'b',
 u'd',
 u'g',
 u'f',
 u's',
 u'S',
 u'v',
 u'z',
 u'Z',
 u'm',
 u'n',
 u'J',
 u'j',
 u'l',
 u'r',
 u'w',
 u'H',
 u'i',
 u'y',
 u'E',
 u'e',
 u'9',
 u'2',
 u'6',
 u'a',
 u'u',
 u'O',
 u'o',
 u'\xea',
 u'\xfb',
 u'\xe2',
 u'\xf4']

In [161]:
temp=diffForms.reset_index()

In [162]:
temp["lexeme"]=temp["lexeme-case"].str.rsplit("-",n=1).str[0]
temp["case"]=temp["lexeme-case"].str.rsplit("-",n=1).str[1]

casesDiff=temp["case"].unique().tolist()
resultTemp={}
for case in casesDiff:
    print "case",case
    paradigmePaire=temp[temp["case"]==case][["value","right","lexeme"]]
    paradigmePaire.rename(columns={"value":case},inplace=True)
    
    paradigmePaire=splitCellMates(splitCellMates(paradigmePaire,case),"right")
    resultTemp[case]=rapports(paradigmePaire)

case ai1P
phonèmes set([u'p'])
phonèmes set([u'u'])
phonèmes set([u'd'])
phonèmes set([u'r'])
phonèmes set([u'y'])
phonèmes set([u'm'])
phonèmes set([u'r'])
phonèmes set([u'6'])
phonèmes set([u'k'])
phonèmes set([u'H'])
phonèmes set([u'i'])
phonèmes set([u'm'])
phonèmes set([u'd'])
phonèmes set([u'E'])
phonèmes set([u'p'])
phonèmes set([u'u'])
phonèmes set([u'r'])
phonèmes set([u'v'])
phonèmes set([u'i'])
phonèmes set([u'm'])
phonèmes set([u'r', u'm', u'g'])
phonèmes set([u'a', u'i', u'y', u'O'])
phonèmes set([u'k', u'b', u'g', u'v'])
phonèmes set([u'E'])
phonèmes set([u't', u'l'])
phonèmes set([u'a'])
phonèmes set([u'm'])
phonèmes set([u'p'])
phonèmes set([u'u'])
phonèmes set([u'd'])
phonèmes set([u'r'])
phonèmes set([u'w'])
phonèmes set([u'a'])
phonèmes set([u'j'])
phonèmes set([u'a'])
phonèmes set([u'm'])
phonèmes set([u'r', u'm', u'g'])
phonèmes set([u'a', u'i', u'y', u'O'])
phonèmes set([u'k', u'b', u'g', u'v'])
phonèmes set([u'6'])
phonèmes set([u't', u'l'])
phonèmes set([u'a'])


phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'k', u'g'])
phonèmes set([u'6'])
phonèmes set([u'z', u't'])
phonèmes set([u'6'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'y', u'r'])
phonèmes set([u'a', u'r'])
phonèmes set([u's'])
phonèmes set([u'w'])
phonèmes set([u'a'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'b', u'E', u'f', u'm', u'p', u's', u'\xe2', u't', u'r'])
phonèmes set([u'g', u'H', u'l', u'p', u'r', u't', u'w', u'z'])
phonèmes set([u'a', u'i', u'E'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'a', u'u', u'r', u'E', u'\xf4'])
phonèmes set([u'r', u'v'])
phonèmes set([u'w'])
phonèmes set([u'a'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'r', u's', u'b', u'E', u'f'])
phonèmes set([u'g', u'j', u'l', u'p', u'r', u't', u'w'])
phonèmes set([u'a', u'E', u'6'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'r'])
phonèmes set([u'a'])
phonèmes set([u's'])
phonèmes set([u'j'])
phonèmes set([u'E'])
phonèmes s

phonèmes set([u'6'])
phonèmes set([u'r'])
phonèmes set([u'\xf4'])
phonèmes set([u'g', u'H', u'l', u'n', u'p', u'r', u't', u'w', u'z'])
phonèmes set([u'a', u'i', u'E'])
phonèmes set([u'r'])
phonèmes set([u'\xf4'])
phonèmes set([u's'])
phonèmes set([u'a'])
phonèmes set([u'j'])
phonèmes set([u'i'])
phonèmes set([u'r'])
phonèmes set([u'\xf4'])
phonèmes set([u'a', u'u', u'r', u'E', u'\xf4'])
phonèmes set([u'r', u'v'])
phonèmes set([u'w'])
phonèmes set([u'a'])
phonèmes set([u'r'])
phonèmes set([u'\xf4'])
case fi3S
phonèmes set([u'H', u'l', u'p', u's', u'r', u't', u'w', u'z'])
phonèmes set([u'a', u'i', u'E'])
phonèmes set([u'j'])
phonèmes set([u'6'])
phonèmes set([u'r'])
phonèmes set([u'a'])
phonèmes set([u'k', u'g'])
phonèmes set([u'E'])
phonèmes set([u'z', u't'])
phonèmes set([u'6'])
phonèmes set([u'r'])
phonèmes set([u'a'])
phonèmes set([u's'])
phonèmes set([u'a'])
phonèmes set([u'j'])
phonèmes set([u'6'])
phonèmes set([u'r'])
phonèmes set([u'a'])
phonèmes set([u'j', u'l', u'p', u's', u't'

phonèmes set([u'j'])
phonèmes set([u'e'])
phonèmes set([u'r', u'm', u'g'])
phonèmes set([u'a', u'y', u'O'])
phonèmes set([u'k', u'b', u'g'])
phonèmes set([u'6'])
phonèmes set([u't'])
phonèmes set([u'j'])
phonèmes set([u'e'])
case ii2S
phonèmes set([u'r'])
phonèmes set([u'a'])
phonèmes set([u's'])
phonèmes set([u'E'])
phonèmes set([u'j'])
phonèmes set([u'E'])
phonèmes set([u'a', u'\xea', u'E', u'd'])
phonèmes set([u'p', u's', u'E', u'v'])
phonèmes set([u'a', u'E', u'O', u'n'])
phonèmes set([u'r', u'\xe2'])
phonèmes set([u't'])
phonèmes set([u'E'])
phonèmes set([u'r', u'm', u'g'])
phonèmes set([u'a', u'i', u'y', u'O'])
phonèmes set([u'k', u'b', u'g', u'v'])
phonèmes set([u'E'])
phonèmes set([u't', u'l'])
phonèmes set([u'E'])
phonèmes set([u'y', u'r'])
phonèmes set([u'a', u'r'])
phonèmes set([u's'])
phonèmes set([u'E'])
phonèmes set([u'j'])
phonèmes set([u'E'])
phonèmes set([u'E'])
phonèmes set([u'm'])
phonèmes set([u'u'])
phonèmes set([u'l'])
phonèmes set([u'i'])
phonèmes set([u'z'])
pho

phonèmes set([u'i'])
phonèmes set([u'z'])
phonèmes set([u'e'])
phonèmes set([u'k', u'\xf4', u'l'])
phonèmes set([u'H', u'f'])
phonèmes set([u'i'])
phonèmes set([u't'])
phonèmes set([u'a', u'\xea', u'E', u'd'])
phonèmes set([u'p', u's', u'E', u'v'])
phonèmes set([u'a', u'E', u'O', u'n'])
phonèmes set([u'r', u'\xe2'])
phonèmes set([u't'])
phonèmes set([u'e'])
phonèmes set([u'E'])
phonèmes set([u'm'])
phonèmes set([u'u'])
phonèmes set([u'l'])
phonèmes set([u'e'])
phonèmes set([u'r', u'm', u'g'])
phonèmes set([u'a', u'i', u'y', u'O'])
phonèmes set([u'k', u'b', u'g', u'v'])
phonèmes set([u'6'])
phonèmes set([u't', u'l'])
phonèmes set([u'e'])
phonèmes set([u'r'])
phonèmes set([u'6'])
phonèmes set([u'v'])
phonèmes set([u'u'])
phonèmes set([u'l'])
phonèmes set([u'e'])
phonèmes set([u'r'])
phonèmes set([u's', u'6'])
phonèmes set([u'w', u'v'])
phonèmes set([u'a', u'u'])
phonèmes set([u'j', u'l'])
phonèmes set([u'e'])
phonèmes set([u'a', u'\xea', u'E', u'd'])
phonèmes set([u'p', u's', u'E', u'v']

phonèmes set([u'E'])
phonèmes set([u'a', u'u', u'r', u'E', u'\xf4'])
phonèmes set([u'r', u'v'])
phonèmes set([u'w'])
phonèmes set([u'a'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'a', u'b', u'E', u'f', u'O', u's', u'r'])
phonèmes set([u'g', u'j', u'l', u'n', u'p', u's', u'r', u't', u'w'])
phonèmes set([u'a', u'E', u'6'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'r'])
phonèmes set([u'a'])
phonèmes set([u's'])
phonèmes set([u'j'])
phonèmes set([u'E'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'g', u'l', u'n', u'p', u's', u'r', u't', u'z'])
phonèmes set([u'E'])
phonèmes set([u'j'])
phonèmes set([u'6'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u'l', u'f'])
phonèmes set([u'i', u'6'])
phonèmes set([u'z', u'j'])
phonèmes set([u'E'])
phonèmes set([u'r', u't'])
phonèmes set([u'6'])
phonèmes set([u'r'])
phonèmes set([u'E'])
phonèmes set([u's'])
phonèmes set([u'a'])
phonèmes set([u'j'])
phonèmes set([u'i'])
phonèmes set([u'r'])
phonème

phonèmes set([u'a', u'k'])
phonèmes set([u's', u'l'])
phonèmes set([u'y', u'i'])
phonèmes set([u'z'])
phonèmes set([u'O', u'f'])
phonèmes set([u'r', u'd'])
phonèmes set([u'i'])
phonèmes set([u't'])
phonèmes set([u'O'])
phonèmes set([u's'])
phonèmes set([u'i'])
phonèmes set([u'z'])
phonèmes set([u'r', u'm', u'g'])
phonèmes set([u'i', u'y', u'O'])
phonèmes set([u'b', u'g', u'v'])
phonèmes set([u'6'])
phonèmes set([u't', u'l'])
phonèmes set([u'e'])
case ppMP
phonèmes set([u'b'])
phonèmes set([u'a'])
phonèmes set([u'j'])
phonèmes set([u'e'])
phonèmes set([u'r', u'm', u'g'])
phonèmes set([u'i', u'y', u'O'])
phonèmes set([u'b', u'g', u'v'])
phonèmes set([u'E'])
phonèmes set([u't', u'l'])
phonèmes set([u'e'])
phonèmes set([u'O'])
phonèmes set([u'k'])
phonèmes set([u's'])
phonèmes set([u'i'])
phonèmes set([u'b'])
phonèmes set([u'a'])
phonèmes set([u'j'])
phonèmes set([u'E'])
phonèmes set([u'O'])
phonèmes set([u's'])
phonèmes set([u'i'])
phonèmes set([u'r', u'm', u'g'])
phonèmes set([u'i', u'y'

In [163]:
resultTemp["pi1S"][0].nbClasse

{'-': 1,
 u'-, -j': 19,
 u'-, -j, vE-al': 1,
 u'-, j-': 27,
 u'-ti': 2,
 u'E.-2.': 2,
 u'O.-o.': 17,
 u'O.-o., -ti': 3,
 u'O..-o..': 1,
 u'O...-o...': 1,
 u'j-': 6,
 u'je-wa': 1}

In [164]:
paradigmePaire

,ps3S,right,lexeme
0,atErmwaj,atErmwa,atermoyer
1,bOZ,boZ,bauger
2,bizOt,bizot,biseauter
3,bwajOt,bwajot,boyauter
4,brHiz,brHis,bruire
5,SOm,Som,chaumer
6,klOstr,klostr,claustrer
7,dOb,dob,dauber
8,dEsErt,dEsErtis,dessertir
9,diplOm,diplom,diplômer


In [165]:
'''Sauvegarder les comparatifs'''
overGeneration.to_csv(path_or_buf=analysisPrefix+"-overGeneration%s.csv"%etapeSwim,encoding="utf8")
underGeneration.to_csv(path_or_buf=analysisPrefix+"-underGeneration%s.csv"%etapeSwim,encoding="utf8")
sameForms.to_csv(path_or_buf=analysisPrefix+"-sameForms%s.csv"%etapeSwim,encoding="utf8")
diffForms.to_csv(path_or_buf=analysisPrefix+"-diffForms%s.csv"%etapeSwim,encoding="utf8")
 

In [166]:
'''Transformer les surabondances en liste'''
diffForms.loc[:,"split-value"]=diffForms.loc[:,"value"].str.split(",")
diffForms.loc[:,"split-right"]=diffForms.loc[:,"right"].str.split(",")

'''Transformer les surabondances en set()'''
diffForms.loc[:,"split-value"]=diffForms.loc[:,"split-value"].apply(set)
diffForms.loc[:,"split-right"]=diffForms.loc[:,"split-right"].apply(set)
    
'''Calculer le nombre de formes (y compris surabondances)'''
nbValues=diffForms["split-value"].str.len().sum()
nbRights=diffForms["split-right"].str.len().sum()

'''Calculer les identités et les inclusions'''
nbIdenticalSets=diffForms[diffForms["split-value"]==diffForms["split-right"]]["split-value"].str.len().sum()
nbIncludedSets=diffForms[diffForms["split-value"]<diffForms["split-right"]]["split-value"].str.len().sum()
nbWrongForms=(nbValues-nbIdenticalSets-nbIncludedSets)
underBonus=(nbRights-nbIdenticalSets-nbIncludedSets)

In [167]:
diffForms[diffForms.index.str.contains("-ii1P")]

,value,right,split,split-value,split-right
lexeme-case,,,,,
craqueter-ii1P,krakEtjô,krak6tjô,[krakEtjô],{krakEtjô},{krak6tjô}
dessertir-ii1P,dEsErtjô,dEsErtisjô,[dEsErtjô],{dEsErtjô},{dEsErtisjô}
dénantir-ii1P,dEnâtjô,dEnâtisjô,[dEnâtjô],{dEnâtjô},{dEnâtisjô}
désassortir-ii1P,dEzasOrtjô,dEzasOrtisjô,[dEzasOrtjô],{dEzasOrtjô},{dEzasOrtisjô}
faillir-ii1P,fajisjô,fajô,[fajisjô],{fajisjô},{fajô}
fleurir-ii1P,fl6risjô,"fl6risjô,flOrisjô",[fl6risjô],{fl6risjô},"{fl6risjô, flOrisjô}"
gobeter-ii1P,gObEtjô,gOb6tjô,[gObEtjô],{gObEtjô},{gOb6tjô}
griveler-ii1P,grivEljô,griv6ljô,[grivEljô],{grivEljô},{griv6ljô}
impartir-ii1P,êpartjô,êpartisjô,[êpartjô],{êpartjô},{êpartisjô}


In [168]:
UG=countSplits(underGeneration)+underBonus
OG=countSplits(overGeneration)
TP=countSplits(sameForms)+nbIdenticalSets+nbIncludedSets
FP=nbWrongForms
resultCharacteristics=(UG,OG,TP,FP)
recall=float(TP)/(UG+TP)
precision=float(TP)/(OG+TP+FP)
fMeasure=2*recall*precision/(recall+precision)
resultMeasures=(precision,recall,fMeasure)
print ("UG",UG ,"OG",OG,"TP",TP,"FP",FP)
print UG+TP
print ("recall", recall, "precision", precision)
print (fMeasure)

('UG', 34908, 'OG', 3676, 'TP', 162207, 'FP', 1078)
197115
('recall', 0.8229054105471425, 'precision', 0.9715262845814292)
0.891061207001


In [169]:
nomFichierResultats=filePrefix+"Longitudinal-X-Resultats-Recalcul.yaml"
if os.path.isfile(nomFichierResultats):
    with open(nomFichierResultats, 'r') as stream:
        resultats=yaml.load(stream)
        if not resultats:
            resultats={}
else:
    resultats={}
    
if casesType:
    sampleExt=casesType
else:
    sampleExt=sampleType
sampleId=samplePrefix.strip("Longitudinal").strip("-")+sampleExt
print resultats

{'Lexique3-00-T150000000-F73539-X-Morphomes': {'Swim1': {'FP': 860, 'nbGoldForms': 197115, 'F-Measure': 0.879581909458241, 'nbInitialForms': 73539, 'OG': 3335, 'Recall': 0.8017553204981863, 'Precision': 0.9741421289133531, 'TP': 158038, 'nbLexemes': 5384, 'UG': 39077}, 'Swim2': {'FP': 1358, 'nbGoldForms': 197115, 'F-Measure': 0.9314862299073368, 'nbInitialForms': 73539, 'OG': 3841, 'Recall': 0.8947517946376481, 'Precision': 0.9713660997532605, 'TP': 176369, 'nbLexemes': 5384, 'UG': 20746}}, 'Lexique3-00-T150000000-F73539-X': {'Swim1': {'FP': 802, 'nbGoldForms': 197115, 'F-Measure': 0.8326274446053942, 'nbInitialForms': 73539, 'OG': 3153, 'Recall': 0.7275600537757146, 'Precision': 0.9731624233212095, 'TP': 143413, 'nbLexemes': 5384, 'UG': 53702}}}


In [170]:
print sampleId
if not sampleId in resultats:
    resultats[sampleId]={}
    
etape=etapeSwim.strip("-")
resultats[sampleId][etape]={}
resultats[sampleId][etape]["nbGoldForms"]=nbGoldForms.item()
resultats[sampleId][etape]["nbInitialForms"]=nbInitialForms.item()
resultats[sampleId][etape]["nbLexemes"]=nbLexemes
resultats[sampleId][etape]["UG"]=UG.item()
resultats[sampleId][etape]["OG"]=OG.item()
resultats[sampleId][etape]["TP"]=TP.item()
resultats[sampleId][etape]["FP"]=FP.item()
resultats[sampleId][etape]["Precision"]=precision
resultats[sampleId][etape]["Recall"]=recall
resultats[sampleId][etape]["F-Measure"]=fMeasure

print resultats

# yaml.safe_dump(resultats, file(nomFichierResultats, 'w'), encoding='utf-8', allow_unicode=True)
with open(nomFichierResultats,"w") as outFile:
    yaml.safe_dump(resultats, outFile, encoding='utf-8', allow_unicode=True)

Lexique3-00-T150000000-F73539-X
{'Lexique3-00-T150000000-F73539-X-Morphomes': {'Swim1': {'FP': 860, 'nbGoldForms': 197115, 'F-Measure': 0.879581909458241, 'nbInitialForms': 73539, 'OG': 3335, 'Recall': 0.8017553204981863, 'Precision': 0.9741421289133531, 'TP': 158038, 'nbLexemes': 5384, 'UG': 39077}, 'Swim2': {'FP': 1358, 'nbGoldForms': 197115, 'F-Measure': 0.9314862299073368, 'nbInitialForms': 73539, 'OG': 3841, 'Recall': 0.8947517946376481, 'Precision': 0.9713660997532605, 'TP': 176369, 'nbLexemes': 5384, 'UG': 20746}}, 'Lexique3-00-T150000000-F73539-X': {'Swim1': {'FP': 802, 'nbGoldForms': 197115, 'F-Measure': 0.8326274446053942, 'nbInitialForms': 73539, 'OG': 3153, 'Recall': 0.7275600537757146, 'Precision': 0.9731624233212095, 'TP': 143413, 'nbLexemes': 5384, 'UG': 53702}, 'Swim2': {'FP': 1078, 'nbLexemes': 5384, 'F-Measure': 0.8910612070007361, 'nbInitialForms': 73539, 'OG': 3676, 'Recall': 0.8229054105471425, 'Precision': 0.9715262845814292, 'TP': 162207, 'nbGoldForms': 197115, '

In [171]:
fin=datetime.datetime.now()
print debut
print fin

2019-09-17 13:34:11.274589
2019-09-17 13:35:16.305675
